### Import Libraries

In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from dataset import *
from PIL import Image
import json
import time

In [ ]:
!pip install unsloth vllm

In [ ]:
from unsloth import FastVisionModel
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

In [ ]:
system_message = """You are a highly advanced Vision Language Model (VLM), specialized in extracting visual data.
Your task is to process and extract meaningful insights from images that are asked in the prompt."""

In [ ]:
from json2xml import json2xml
from json2xml.utils import readfromstring
from lxml import etree
import base64


def format_data(sample, train_type: str):
    pil_image = Image.open(sample.image_path)

    field_names = set([entity.label for entity in sample.entities])
    if train_type == "xml":
        xml_fields = "".join([f"<{field}>..</{field}>" for field in field_names])
        output_format = f"<kie>{xml_fields}</kie>"
        prompt = "Extract the following {fields} from the above document. If a field is not present, return ''. Return the output in a valid XML format like {output_format}" \
            .format(
                fields = list(field_names),
                output_format = output_format
            )
    else:
        output_format = {field: ".." for field in field_names}

        prompt = "Extract the following {fields} from the above document. If a field is not present, return ''. Return the output in a valid JSON format like {output_format}" \
            .format(
                fields = list(field_names),
                output_format = output_format
            )

    if train_type == "normal":
        conversation = [
            {
                "role": "system",
                "content": [{"type": "text", "text": system_message}]
            },
            {
                "role": "user",
                "content": [
                    { "type": "image", "image": pil_image },
                    { "type": "text", "text": prompt }
                ]
            },
            {
                "role": "assistant",
                "content": [{
                    "type": "text",
                    "text": json.dumps(sample.to_json("kie"))
                }]
            }
        ]
    elif train_type == "no-prompt":
        conversation = [
            {
                "role": "user",
                "content": [
                    { "type": "image", "image": pil_image }
                ]
            },
            {
                "role": "assistant",
                "content": [{
                    "type": "text",
                    "text": json.dumps(sample.to_json("kie"))
                }]
            }
        ]
    elif train_type == "xml":
        label = json2xml.Json2xml(
            data=readfromstring(json.dumps(sample.to_json("kie"))),
            wrapper="kie",
            pretty=False,
            attr_type=False
        ).to_xml()
        label = etree.tostring(
            etree.fromstring(label),
            encoding="unicode",
            pretty_print=False
        )

        conversation = [
            {
                "role": "system",
                "content": [{"type": "text", "text": system_message}]
            },
            {
                "role": "user",
                "content": [
                    { "type": "image", "image": pil_image },
                    { "type": "text", "text": prompt }
                ]
            },
            {
                "role": "assistant",
                "content": [{
                    "type": "text",
                    "text": label
                }]
            }
        ]
    else:
        raise Exception(f"{train_type} value error")

    return conversation

In [ ]:
train_type = "normal"

train_dataset = [format_data(sample, train_type) for sample in SROIE(tasks=["kie"], split="train")]
test_dataset = [format_data(sample, train_type) for sample in SROIE(tasks=["kie"], split="test")]

train_dataset[0]

### Training Pipeline

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-3B-Instruct-bnb-4bit",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [ ]:
if train_type == "xml":
    for x in ['company', 'date', 'address', 'total']:
        tokenizer.tokenizer.add_tokens([f"<doc_{x}>", f"</doc_{x}>"])

    tokenizer.tokenizer.add_tokens(["<kie>", "</kie>"])
    model.resize_token_embeddings(len(tokenizer.tokenizer))

    model.config.pad_token_id = tokenizer.tokenizer.pad_token_id
    model.config.decoder_start_token_id = tokenizer.tokenizer.convert_tokens_to_ids("<kie>")
    model.config.eos_token_id = tokenizer.tokenizer.convert_tokens_to_ids("</kie>")

In [ ]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    #use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
def json_parsable(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    results = []
    for response in responses:
        try:
            json.loads(response)
            results.append(2.0)
        except json.JSONDecodeError:
            results.append(0.0)
    return results

def all_fields_present(completions, **kwargs) -> list[float]:
    labels = {"address", "date", "company", "total"}
    responses = [completion[0]['content'] for completion in completions]
    results = []
    for response in responses:
        try:
            response_dict = json.loads(response)
            results.append(2.0 if labels.issubset(response_dict.keys()) else 0.0)
        except json.JSONDecodeError:
            results.append(0.0)
    return results

def correct_labels(completions, answer, **kwargs) -> list[float]:
    labels = {"address", "date", "company", "total"}
    results = []
    for completion, gt in zip(completions, answer):
        score = 0.0
        try:
            completion_dict = json.loads(completion[0]['content'])
            gt_dict = json.loads(gt)
            for label in labels:
                pred = completion_dict.get(label, None)
                gt_value = gt_dict.get(label, None)

                if pred == gt_value and pred is not None:
                    score += 1.0
                else:
                    score -= 2.0
            results.append(score)
        except json.JSONDecodeError:
            results.append(0.0)
    return results

# TODO: aggiungere funzione per calcolare editdistance sui campi

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        json_parsable,
        all_fields_present,
        correct_labels
    ],
    args = training_args,
    train_dataset = train_dataset,
)
trainer.train()

In [ ]:
trainer_stats = trainer.train()